#### Initial configurations

In [36]:
import pandas as pd
import re

In [37]:
df = pd.read_csv('inputs/data_storeproduct_202208111119.csv', usecols = ['id', 'name'])

In [38]:
df.shape

(820512, 2)

In [39]:
df.drop_duplicates(subset = 'name', inplace = True)

In [40]:
df.shape

(306052, 2)

In [41]:
df['name'] = df['name'].str.upper()

In [42]:
df['clean'] = df['name']

---

### Removes parentheses

In [43]:
df['clean'] = df['clean'].apply(lambda x: re.sub(r'\([^()]*\)', '', x) if x else x)

In [44]:
# # Tested Items
# df.loc[[743024, 270107, 210205, 295409, 295406, 764575, 208980, 205859, 764574, 796994, 744286]]

---

### Removes initial digits

In [45]:
def clean_initial_digits(name):
    try:
        word = name.split()[0]
        digits = sum(l.isdigit() for l in word)
        
        if digits >= len(word) - digits:
            return name.replace(word, '')
        else:
            return name
    except:
        return name

In [46]:
df['clean'] = df['clean'].apply(lambda x: clean_initial_digits(x))

In [47]:
# # Tested Items
# df.loc[[638551, 337465, 337317, 32411, 557546]]

---

### Removes initial characters

In [48]:
def clean_initial_characters(name):
    chars = ['!', '%', '*', '+', ',', '-', '.', '/', '<', '=']
    to_remove = []

    try:
        word = name.split()[0]
        for letter in word:
            if letter in chars:
                to_remove.append(letter)
        
        if len(to_remove) > 0:
            cleaned = ''.join([w for w in word if w not in to_remove])
            name = name.split()[1:]
            name.insert(0, cleaned)
            
            return ' '.join(name)
        else:
            return name
    except:
        return name

In [49]:
df['clean'] = df['clean'].apply(lambda x: clean_initial_characters(x))

In [50]:
# # Tested Items
# df.loc[[481395, 295715, 755477, 178663, 484751, 239320, 28118, 280929, 170862, 106470]]

---

### Removes units

In [51]:
def clean_units(name):
    units = [
        'U', 'G', 'V', 'P', 'L', 'M', 
        'GR', 'KG', 'ML', 'UN', 'CM', 'MG', 'CP', 'MM', 'CM', 'PC', 
        'CPS', 'CPR', 'VOL', 'CAP', 'UND', 'GRS', 
        'CAPS', 'UNID', 'UNDS', 'CPRS', 'COMP', 
        'UNID.', 'SACHES' 
    ]
    to_remove = []
    words = name.split()

    for word in words:
        for unit in units:
            if word == unit:
                to_remove.append(word)
                
                try:
                    previous_word = words[words.index(word) - 1]
                    digits = sum(c.isdigit() for c in previous_word)

                    if digits >= len(previous_word) - digits:
                        to_remove.append(previous_word)
                except:
                    pass

            if word[-len(unit):] == unit:
                qty = word[:-len(unit)]
                digits = sum(c.isdigit() for c in qty)
                
                if digits >= len(qty) - digits:
                    to_remove.append(word)

    return ' '.join([word for word in words if word not in to_remove])

In [52]:
df['clean'] = df['clean'].apply(lambda x: clean_units(x))

In [53]:
# # Tested Items
# df.loc[[
#     363199, 635298, 40689,  740467, 213141, 679374, 288780, 810939, 510231, 648258, 
#     704156, 178001, 413533, 785982, 340359, 211457, 565330, 739959, 491207, 598294, 
#     212840, 585916, 575283, 786004, 545491, 646295, 734862, 353142, 51607,  624800, 
#     256973, 713142, 536849, 165346, 214062, 685358, 520570, 120444, 685071, 712342, 
#     484879, 685358, 765239, 658096, 729729, 545046, 507365, 444408, 29568,  168168, 
#     220269, 238971, 487450, 726061, 696019, 122934, 649598, 350363
# ]]

---

### Removes abbreviations

In [54]:
def clean_abbrs(name):
    abbrs = ['REF', 'REF.', 'C/', 'S/', 'P/', 'FPS']
    to_remove = []
    words = name.split()

    for word in words:
        if word in abbrs:
            to_remove.append(word)

            try:
                to_remove.append(words[words.index(word) + 1])
            except:
                pass
            
        elif word.startswith(tuple(abbrs)):
            for abbr in abbrs:
                if abbr == 'REF' == word[:3] and word[3] != '.' and not word[len(abbr):][0].isdigit():
                    break
                else:
                    to_remove.append(word)

    return ' '.join([word for word in words if word not in to_remove])

In [55]:
df['clean'] = df['clean'].apply(lambda x: clean_abbrs(x))

In [56]:
# # Tested Items
# df.loc[[
#     507889, 308443, 814522, 708421, 800889, 773057, 671045, 583425, 582548, 700178, 
#     318441, 318442, 214566, 632491, 480583, 733038, 407674, 658417, 310619, 648318
# ]]

---

### Removes initial digits and characters

In [57]:
df['clean'] = df['clean'].apply(lambda x: clean_initial_digits(x))

In [58]:
df['clean'] = df['clean'].apply(lambda x: clean_initial_characters(x))

---

### Removes isolated words

In [59]:
def clean_isolated(name):
    isolated = [
        'CR', 'COMP.', 'COMPR', 'COMPRIMIDOS', 
        'CAP.', 'CAPS.', 'CAPSULA', 'CAPSULAS',
        'CX.', 'CX', 'CAIXA'
    ]
    to_remove = []
    words = name.split()

    for word in words:
        if word in isolated:
            to_remove.append(word)

    return ' '.join([word for word in words if word not in to_remove])

In [60]:
df['clean'] = df['clean'].apply(lambda x: clean_isolated(x))

In [61]:
# Tested Items
df.loc[[413099, 208413, 291647, 239129, 416768, 504305, 173425, 804124, 239195, 175181, 634522]]

,id,name,clean
413099,475913,HERCAP 5MG CR 30G,HERCAP
208413,312547,AAS 100MG C/120 COMP.,AAS
291647,221052,"ABLOK PLUS 25+12,5MG CX 60 COMPR",ABLOK PLUS 60
239129,230169,ACICLOVIR 200MG C/25 COMPRIMIDOS,ACICLOVIR
416768,201238,DOSS 50000 UI CX C/8 CAP. GELATINOSAS,DOSS 50000 UI GELATINOSAS
504305,425358,ADDERA 60.000UI C/2 CAPS.,ADDERA 60.000UI
173425,403301,ADEFORTE CAPSULA CX C 60,ADEFORTE C 60
804124,366740,ACIDO HIALURONICO 60 CAPSULAS,ACIDO HIALURONICO 60
239195,436964,GLICLAZIDA 60MG - CX. C/ 60 COMP LIB PROL,GLICLAZIDA - PROL
175181,1022435,AAS 100MG CX 30 COMP,AAS


---

### Creates a CSV file with cleaned names

In [62]:
df['clean'] = df['clean'].str.strip()

In [63]:
df['had_changes'] = df['clean'] != df['name']

In [64]:
df['num_words'] = df['clean'].apply(lambda x: len(str(x).split()))

In [65]:
df.sort_values(by = 'clean', inplace = True)

In [66]:
df.to_csv('outputs/clean.csv', index = False)

---

### Metrics

In [67]:
print(f"Items\n{'-' * len('Items')}")
print(f"Total: {df.shape[0]}")
print(f"Changed: {len(df[df['had_changes'] == True])}")
print(f"Not Changed: {len(df[df['had_changes'] == False])}")

# TOTAL     CHANGED     NOT CHANGED
# 306.052   36.829      269.223
# 306.052   91.447      214.605
# 306.052   232.885     73.167
# 306.052   234.920     71.132
# 306.052   236.925     69.127

Items
-----
Total: 306052
Changed: 236925
Not Changed: 69127


In [68]:
print(f"Unique Items\n{'-' * len('Unique Items')}")
print(f"Original dataset: {df['name'].nunique()}")
print(f"Cleaned dataset: {df['clean'].nunique()}")
print(f"Unified items: {df['name'].nunique() - df['clean'].nunique()}")

# ORIGINAL      CLEANED     UNIFIED
# 306.051       303.710     2.341
# 306.051       297.114     8.937
# 306.051       259.696     46.355
# 306.051       256.832     49.219
# 306.051       252.891     53.160

Unique Items
------------
Original dataset: 306051
Cleaned dataset: 252891
Unified items: 53160


---

In [69]:
# df[df['clean'].str.contains('|'.join(unique_))]
# df[df['clean'].str.contains(' UNID. ')]

---

In [70]:
# dict_ = {
#     'TX': 'TAXA',
#     'ABS': 'ABSORVENTE',
#     'ESP': 'ESPARADRAPO',
#     'SAB': 'SABONETE',
#     'DES': 'DESODORANTE',
#     'ESC': 'ESCOVA',
#     'LOC': 'LOCAO',
#     'PERF': 'PERFUME',
#     'TRAD': 'TRADICIONAL',
#     'CURAT': 'CURATIVO',
# }